# Indonesian Text Classification

This is a text classification  of indonesian corpus using several different technique such as Naive Bayes, SVM, Random Forest, Convolition Neural Network (CNN), LSTM or GRU. An indonesian [pre-trained word vectors](https://fasttext.cc/docs/en/pretrained-vectors.html) from FastText has ben also used in our Neural Network models.
We use [Word Bahasa Indonesia Corpus and Parallel English Translation](https://www.panl10n.net/english/outputs/Indonesia/BPPT/0902/BPPTIndToEngCorpusHalfM.zip) dataset from PAN Localization.
It contains 500,000 words from various online sources translated into English.
For our text classification, we use only the indonesian part.
The corpus has 4 classes:
  - 0: Economy
  - 1: International
  - 2: Science
  - 3: Sport
 
Originally each class is in separate file, we combine, randomize and split it to train and test file with 90:10.  



In [0]:
from sklearn import model_selection, preprocessing
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import tensorflow as tf
#import pandas as pd, xgboost, numpy, textblob, string
import pandas as pd, xgboost, numpy, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing.sequence import pad_sequences
from pathlib import Path

import os
import numpy as np
#import ntlk

In [0]:
# Parameters
LMDATA = Path('/content/drive/My Drive/lmdata')
params = {'batch_size': 1024,
          'n_classes': 2,
          'max_len': 100,
          'n_words': 50000,
          'shuffle': True}
try:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  strategy=tf.contrib.tpu.TPUDistributionStrategy(
      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
except KeyError:
  TPU_WORKER = None

np.random.seed(seed=10)

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pwd
print(LMDATA)
!ls -lh "$LMDATA"
!ls -lh "$LMDATA/BPPTIndToEngCorpus"

/content
/content/drive/My Drive/lmdata
total 3.0G
-rw------- 1 root root 4.3M Apr 21 23:28 amazon-review.csv
-rw------- 1 root root  58M Oct  8 16:03 bard-1.h5
-rw------- 1 root root  31M Oct 11 19:03 bard.h5
drwx------ 2 root root 4.0K Oct  9 15:04 BPPTIndToEngCorpus
-rw------- 1 root root  25M Oct  9 16:46 wiki.id.10K.vec
-rw------- 1 root root 755M Oct  9 16:35 wiki.id.300K.vec
-rw------- 1 root root  22M Oct  8 13:31 wiki-news-300d-10K.vec
-rw------- 1 root root 2.2G Mar 14  2018 wiki-news-300d-1M.vec
total 7.1M
-rw------- 1 root root 3.6M Oct 11 10:45 bppt_panl.csv
-rw------- 1 root root 362K Oct 11 10:45 bppt_panl_test.csv
-rw------- 1 root root 3.2M Oct 11 10:45 bppt_panl_train.csv


In [0]:
train_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_train.csv')
train_df.columns = ['label', 'text']
test_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_test.csv')
test_df.columns = ['label', 'text']

In [6]:
print(train_df.head())
#print(train_df['label'][:10].values)
#print(train_df['text'][:10].values)

   label                                               text
0      0  Pertumbuhan ekonomi 2007 yang diproyeksikan me...
1      3  Pelatih Real Bernd Schuster harus mengeluarkan...
2      2  Laporan itu adalah pengumuman kedua dari badan...
3      0  Lonjakan laba bersih tersebut, selain didorong...
4      3  LeBron James menyumbang 24 poin, 11 assist dan...


In [0]:
!set |grep -i tpu|grep -v grep

In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], train_df['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [9]:
print(train_x[:5])
print(train_y[:5])
print(valid_x[:5])
print(valid_y[:5])


19555    Layanan fantastis ini membutuhkan waktu lama u...
9774     Gasparotto memimpin lomba keseluruhan disusul ...
10459    Pereli Prancis itu unggul dua menit 33,2 detik...
19916    Karena itu Antam juga melakukan diversifikasi ...
15343    Para kapitalis ini menginginkan pemerintah Ind...
Name: text, dtype: object
[2 3 3 0 0]
3081     Menurut Shahab, peralatan Rig pada BJP-1R1 sud...
1601     Anak pepsis memakan daging tarantula dan berli...
20221    Dalam hal kepemilikan asset, maka sektor-sekto...
2118     Ilmuwan telah menemukan informasi genetik yang...
17469    Anda menyesuaikan dengan itu dan lawan juga sama.
Name: text, dtype: object
[0 2 0 2 3]


In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(train_df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train_df['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [12]:
# load the pre-trained word-embedding vectors
max_words = params['n_words']
embeddings_index = {}
for i, line in enumerate(open('drive/My Drive/lmdata/wiki.id.300K.vec')):
  if i%50000 == 0:
    print(i)
  values = line.split()
  try:
    embeddings_index[" ".join(values[0:-300])] = numpy.asarray(values[-300:], dtype='float32')
  except ValueError:
    print("Values: {}: {}".format(i, values))

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(train_df['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), 
                                     maxlen=params['max_len'])
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), 
                                     maxlen=params['max_len'])

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i>=max_words:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

0
50000
100000
150000
200000
250000
300000


## Konventional Machine Learning

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, 
                is_neural_net=False, epochs=1):
    # fit the training dataset on the classifier
    if is_neural_net:
      classifier.fit(feature_vector_train, label, epochs=epochs)
    else:
      classifier.fit(feature_vector_train, label)   
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
      predictions = [int(round(p[0])) for p in predictions]
      #predictions = predictions.argmax(axis=-1)

    print(" predictions:", predictions[:20])
    print("ground truth:", valid_y[:20])
    
    return metrics.accuracy_score(predictions, valid_y)

### Naive Bayes

In [14]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

 predictions: [0 2 0 2 3 0 0 2 0 1 0 2 2 0 1 1 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
NB, Count Vectors:  0.9269195189639223
 predictions: [0 2 0 2 3 0 1 2 0 1 0 2 2 0 1 1 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
NB, WordLevel TF-IDF:  0.9161887141535615
 predictions: [0 2 0 2 1 0 1 2 0 1 0 2 2 0 1 0 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
NB, N-Gram Vectors:  0.7822386679000926
 predictions: [0 2 0 2 2 0 0 2 2 1 2 2 0 0 1 2 3 2 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
NB, CharLevel Vectors:  0.8432932469935245


### Linear Classifier

In [15]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

 predictions: [0 2 0 2 2 0 0 2 2 1 0 2 2 0 0 1 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
LR, Count Vectors:  0.9265494912118409
 predictions: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 1 0 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
LR, WordLevel TF-IDF:  0.9178538390379278
 predictions: [0 2 2 2 1 0 1 2 2 1 0 2 2 0 1 0 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
LR, N-Gram Vectors:  0.8085106382978723
 predictions: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 1 2 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
LR, CharLevel Vectors:  0.8888066604995375


### SVM

In [16]:
# SVM on Ngram Level TF IDF Vectors
# It seems there is something wrong here, since the predictions are always 0
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


 predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
SVM, N-Gram Vectors:  0.25827937095282144


### Random Forest

In [20]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

 predictions: [0 2 0 2 2 0 0 2 2 1 0 2 2 0 1 0 3 1 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
RF, Count Vectors:  0.8392229417206291
 predictions: [0 2 0 2 1 0 0 2 0 1 0 2 3 0 0 0 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
RF, WordLevel TF-IDF:  0.8338575393154487


###  Extreme Gradient Boosting

In [18]:
# Extreme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extreme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extreme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [0 2 2 2 2 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
Xgb, Count Vectors:  0.808695652173913


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [0 2 2 2 2 0 0 2 2 1 0 2 2 0 0 2 3 2 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
Xgb, WordLevel TF-IDF:  0.8070305272895467
 predictions: [0 2 0 2 2 0 1 2 2 2 0 2 2 0 1 2 3 2 1 3]
ground truth: [0 2 0 2 3 0 0 2 2 1 0 2 2 0 0 2 3 0 1 3]
Xgb, CharLevel Vectors:  0.8201665124884366


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Neural Network

In [0]:
from tensorflow.keras.utils import Sequence, to_categorical

def tokenize(texts, n_words=1000):
    tokenizer = Tokenizer(num_words=n_words)
    tokenizer.fit_on_texts(texts)
    return tokenizer
  
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, texts, labels, tokenizer, batch_size=32, max_len=100,
                 n_classes=2, n_words=1000, shuffle=True):
        'Initialization'
        self.max_len = max_len
        self.batch_size = batch_size
        self.texts = texts
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle

        self.tokenizer = tokenizer
        self.steps_per_epoch = int(np.floor(self.texts.size / self.batch_size))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.steps_per_epoch

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        texts = np.array([self.texts[k] for k in indexes])
        sequences = self.tokenizer.texts_to_sequences(texts)
        X = pad_sequences(sequences, maxlen=self.max_len)
        y = np.array([to_categorical(self.labels[k], 4) for k in indexes])

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.texts.size)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [0]:
# Create data generator
training_generator = DataGenerator(train_x.values, train_y, token, **params)
valid_generator = DataGenerator(valid_x.values, valid_y, token,  **params)

In [0]:
def tpu_wrapper(func):
  if TPU_WORKER is not None:
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
    return tpu_model
  else:
    return func

In [0]:
# This model doesn't work with current dataset
def create_simple_model(input_length=100):
    # create input layer 
    input_layer = layers.Input((input_length, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu", name="D1")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(4, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    return classifier 

#classifier = create_simple_model(xtrain_tfidf_ngram.shape[1])
#accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 
#                       is_neural_net=True, epochs=1)
#print("NN, Ngram Level TF IDF Vectors",  accuracy)

In [0]:
%%time

#NN, Ngram Level TF IDF Vectors
tf.keras.backend.clear_session()
classifier = create_simple_model(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)

In [25]:
xtrain_tfidf_ngram.shape

(16214, 5000)

In [0]:
"""
def to_tpu(func):
    def wrapper(*args, **kwargs):
      print("TPU Wrapper start")
      print(func)
      if TPU_WORKER is not None:
        tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
        print("TPU exist")
        return tpu_model(*args, **kwargs)
      else:
        print("TPU not exist")
        return func(*args, **kwargs)
    print("TO_TPU")
    return wrapper
"""

### CNN

In [0]:
def create_cnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, 
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.3)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [25]:
%%time

tf.keras.backend.clear_session()
classifier = create_cnn(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)


Epoch 1/20
15/15 [==============================] - 4s 289ms/step - loss: 1.1967 - acc: 0.4941 - val_loss: 0.7733 - val_acc: 0.8369
Epoch 2/20
15/15 [==============================] - 2s 114ms/step - loss: 0.6243 - acc: 0.8046 - val_loss: 0.4103 - val_acc: 0.8627
Epoch 3/20
15/15 [==============================] - 2s 138ms/step - loss: 0.4095 - acc: 0.8594 - val_loss: 0.3286 - val_acc: 0.8775
Epoch 4/20
15/15 [==============================] - 2s 126ms/step - loss: 0.3417 - acc: 0.8813 - val_loss: 0.2932 - val_acc: 0.8906
Epoch 5/20
15/15 [==============================] - 2s 141ms/step - loss: 0.3069 - acc: 0.8923 - val_loss: 0.2733 - val_acc: 0.8982
Epoch 6/20
15/15 [==============================] - 2s 122ms/step - loss: 0.2855 - acc: 0.8998 - val_loss: 0.2594 - val_acc: 0.9047
Epoch 7/20
15/15 [==============================] - 2s 126ms/step - loss: 0.2601 - acc: 0.9102 - val_loss: 0.2487 - val_acc: 0.9064
Epoch 8/20
15/15 [==============================] - 2s 129ms/step - loss: 0.

In [0]:

classifier.save_weights(str(LMDATA/'cnn.h5'), overwrite=True)

INFO:tensorflow:Copying TPU weights to the CPU


In [26]:
# predict the labels on test dataset

test_x = test_df['text'].values
sequences = token.texts_to_sequences(test_x)
test_x_seq = pad_sequences(sequences, maxlen=params['max_len'])
#print(valid_x_seq[:5])
classifier = create_cnn(input_length=params['max_len'])
classifier.load_weights(str(LMDATA/'bard.h5'))
# TPU can be enabled here if we need  
# classifier = tpu_wrapper(classifier)
labels = np.array([list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values])
score = classifier.evaluate(test_x_seq, labels, verbose=1)
print('Loss for final step: {}, accuracy: {}'.format(score[0], score[1]))




2402/2402 [==============================] - 0s 205us/step
Loss for final step: 0.20907926316265263, accuracy: 0.925895087427144


In [0]:
"""
# Just for testing
labels = [list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values]
#print(labels[:5])
print(np.array(labels)[:5])
print(test_df['label'].values)
print(to_categorical(3, 4))
"""

In [27]:
prediction = classifier.predict(test_x_seq, verbose=1)
#print(prediction)
print('Predictions for final step: {}'.format(np.argmax(prediction, axis=1)[:10]))
print(test_x[:10])

2402/2402 [==============================] - 0s 81us/step
Predictions for final step: [0 0 2 3 1 0 0 1 2 2]
['Paradoksnya di sisi lain, sinyal akan diakuinya keberadaan lembaga keuangan mikro non formal juga membuat kegelisahan bagi pelakunya.'
 'Menurut dia, harga minyak mentah dunia saat ini berada di posisi 126 dolar AS per barel sedikit melemah dibanding hari sebelumnya yang mencapai 127 dolar AS lebih.'
 'Pengkajian ini memberikan konfirmasi bahwa kita menghadapi sejenis materi yang berbeda sama sekali, tak seperti yang kita bayangkan.'
 'Gol akhir Marco Borriello membuat Genoa mendapat satu angka setelah striker asal Honduras, David Suazo, mencetak angka pada menit ke-12 untuk Inter, yang pemain tengahnya dari Portugal, Pele, dikeluarkan dari lapangan karena mendapat kartu kuning kedua sebelum turun minum.'
 'Namun, bandar udara itu ditutup sebagai langkah pencegahan.'
 'Departemen Keuangan telah menetapkan 16 calon agen penjual obligasi negara ritel ORI pada 2007 melalui SK Dirj

In [36]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13335204     1260560     8939392         804     3135252    12365508
Swap:             0           0           0


In [0]:

#print('Accuracy ', score[1])
"""
# First, run the seed forward to prime the state of the model.
#prediction_model.reset_states()
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)

predictions = prediction_model.predict(valid_x_seq)

print("predictions", predictions[:20])

predictions = tpu_model.predict(valid_x_seq)

predictions = [int(round(p[0])) for p in predictions]
#predictions = predictions.argmax(axis=-1)

print("predictions", predictions[:20])
print("valid_y", valid_y[:20])

return metrics.accuracy_score(predictions, valid_y)
"""

In [39]:
print(prediction)
print(np.argmax(prediction, axis=1))


[[9.9899882e-01 4.6373406e-04 5.3280470e-04 4.7004282e-06]
 [9.9977285e-01 1.3875878e-04 7.5860429e-05 1.2497356e-05]
 [1.0611779e-01 4.8861925e-02 8.3545119e-01 9.5691448e-03]
 ...
 [1.1416495e-02 9.8778975e-01 4.8050229e-04 3.1316865e-04]
 [1.4843496e-03 5.0487202e-02 4.8084562e-03 9.4321996e-01]
 [1.4797889e-02 2.0483567e-03 9.8280776e-01 3.4594335e-04]]
[0 0 2 ... 1 3 2]


### Kim Yoon’s CNN

In [0]:
# The following model is similar to 
# Kim Yoon’s Convolutional Neural Networks for Sentence Classification
# (https://arxiv.org/abs/1408.5882)
#
# Model Hyperparameters
embedding_dim = 300
filter_sizes = (3, 5, 7)
num_filters = 100
dropout_prob = (0.5, 0.5)
hidden_dims = 50

def create_cnn_kimyoon(input_length=100):  
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, embedding_dim, 
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(dropout_prob[0])(embedding_layer)
    
    conv_array = []
    for sz in filter_sizes:
        conv = layers.Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(embedding_layer)
        conv = layers.MaxPooling1D(pool_size=2)(conv)
        conv = layers.Flatten()(conv)
        conv_array.append(conv)
    
    layer = layers.Concatenate()(conv_array) if len(conv_array) > 1 else conv_array[0]
    
    layer = layers.Dropout(dropout_prob[1])(layer)
    layer = layers.Dense(hidden_dims, activation="relu")(layer)
    output_layer = layers.Dense(4, activation="softmax")(layer)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [27]:
%%time

tf.keras.backend.clear_session()
classifier = create_cnn_kimyoon(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'cnn_kimyoon.h5'), overwrite=True)


Epoch 1/20
15/15 [==============================] - 6s 385ms/step - loss: 1.0868 - acc: 0.5242 - val_loss: 0.5332 - val_acc: 0.8217
Epoch 2/20
15/15 [==============================] - 4s 239ms/step - loss: 0.4959 - acc: 0.8238 - val_loss: 0.3524 - val_acc: 0.8695
Epoch 3/20
15/15 [==============================] - 4s 238ms/step - loss: 0.4111 - acc: 0.8513 - val_loss: 0.3162 - val_acc: 0.8943
Epoch 4/20
15/15 [==============================] - 3s 226ms/step - loss: 0.3709 - acc: 0.8645 - val_loss: 0.2937 - val_acc: 0.9002
Epoch 5/20
15/15 [==============================] - 4s 239ms/step - loss: 0.3413 - acc: 0.8755 - val_loss: 0.2892 - val_acc: 0.9021
Epoch 6/20
15/15 [==============================] - 3s 226ms/step - loss: 0.3251 - acc: 0.8835 - val_loss: 0.2829 - val_acc: 0.9043
Epoch 7/20
15/15 [==============================] - 3s 231ms/step - loss: 0.3097 - acc: 0.8870 - val_loss: 0.2748 - val_acc: 0.9072
Epoch 8/20
15/15 [==============================] - 3s 227ms/step - loss: 0.

### RNN-LSTM

In [0]:
def create_rnn_lstm(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.2)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)
    #lstm_layer = layers.LSTM(100)(lstm_layer)
    #lstm_layer = layers.LSTM(100, return_sequences=True)(lstm_layer)
    #lstm_layer = layers.TimeDistributed(layers.Dense(100))(lstm_layer)
    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.2)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [29]:
%%time

# RNN-LSTM, Word Embeddings
# Speed comparison between CPU vs TPU
# First we test CPU with 2 epochs
tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
#classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=2
)

Epoch 1/2
15/15 [==============================] - 9s 627ms/step - loss: 1.1598 - acc: 0.5539 - val_loss: 0.6460 - val_acc: 0.8254
Epoch 2/2
15/15 [==============================] - 6s 422ms/step - loss: 0.4805 - acc: 0.8472 - val_loss: 0.3420 - val_acc: 0.8754
CPU times: user 20.4 s, sys: 2.24 s, total: 22.6 s
Wall time: 16.4 s


In [30]:
%%time

# RNN-LSTM, Word Embeddings
# We test now the TPU
# The result is:
# CPU: 65s/epoch
# TPU: 3.15s/epoch
tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
15/15 [==============================] - 9s 632ms/step - loss: 1.1334 - acc: 0.5769 - val_loss: 0.6212 - val_acc: 0.8119
Epoch 2/20
15/15 [==============================] - 6s 426ms/step - loss: 0.4754 - acc: 0.8382 - val_loss: 0.3575 - val_acc: 0.8678
Epoch 3/20
15/15 [==============================] - 6s 422ms/step - loss: 0.3443 - acc: 0.8812 - val_loss: 0.2856 - val_acc: 0.8977
Epoch 4/20
15/15 [==============================] - 6s 426ms/step - loss: 0.2977 - acc: 0.8954 - val_loss: 0.2644 - val_acc: 0.9029
Epoch 5/20
15/15 [==============================] - 6s 426ms/step - loss: 0.2859 - acc: 0.8988 - val_loss: 0.2579 - val_acc: 0.9059
Epoch 6/20
15/15 [==============================] - 6s 417ms/step - loss: 0.2666 - acc: 0.9070 - val_loss: 0.2463 - val_acc: 0.9119
Epoch 7/20
15/15 [==============================] - 6s 415ms/step - loss: 0.2505 - acc: 0.9117 - val_loss: 0.2372 - val_acc: 0.9119
Epoch 8/20
15/15 [==============================] - 6s 417ms/step - loss: 0.

### RNN-GRU

In [0]:
def create_rnn_gru(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [32]:
%%time

# RNN-GRU, Word Embeddings
tf.keras.backend.clear_session()
classifier = create_rnn_gru(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
15/15 [==============================] - 8s 559ms/step - loss: 1.2750 - acc: 0.4177 - val_loss: 1.0444 - val_acc: 0.6666
Epoch 2/20
15/15 [==============================] - 5s 363ms/step - loss: 0.8752 - acc: 0.6757 - val_loss: 0.5467 - val_acc: 0.8154
Epoch 3/20
15/15 [==============================] - 5s 353ms/step - loss: 0.4949 - acc: 0.8301 - val_loss: 0.3342 - val_acc: 0.8836
Epoch 4/20
15/15 [==============================] - 5s 357ms/step - loss: 0.3629 - acc: 0.8741 - val_loss: 0.2751 - val_acc: 0.8992
Epoch 5/20
15/15 [==============================] - 5s 355ms/step - loss: 0.3151 - acc: 0.8926 - val_loss: 0.2546 - val_acc: 0.9062
Epoch 6/20
15/15 [==============================] - 5s 352ms/step - loss: 0.2959 - acc: 0.8983 - val_loss: 0.2453 - val_acc: 0.9086
Epoch 7/20
15/15 [==============================] - 5s 363ms/step - loss: 0.2830 - acc: 0.9013 - val_loss: 0.2436 - val_acc: 0.9115
Epoch 8/20
15/15 [==============================] - 5s 365ms/step - loss: 0.

### Biderectional RNN

In [0]:
# RNN-Bidirectional, Word Embeddings
# It doesn't work with TPU, but it works on CPU/GPU
def create_bidirectional_rnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [34]:
%%time

# RNN-Bidirectional, Word Embeddings
# It doesn't work with TPU, but it works on CPU/GPU
tf.keras.backend.clear_session()
classifier = create_bidirectional_rnn(input_length=params['max_len'])
#classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
15/15 [==============================] - 14s 904ms/step - loss: 1.2845 - acc: 0.4305 - val_loss: 1.0628 - val_acc: 0.6744
Epoch 2/20
15/15 [==============================] - 9s 629ms/step - loss: 0.8630 - acc: 0.6880 - val_loss: 0.5020 - val_acc: 0.8223
Epoch 3/20
15/15 [==============================] - 9s 623ms/step - loss: 0.4589 - acc: 0.8399 - val_loss: 0.3252 - val_acc: 0.8834
Epoch 4/20
15/15 [==============================] - 9s 624ms/step - loss: 0.3468 - acc: 0.8801 - val_loss: 0.2694 - val_acc: 0.9000
Epoch 5/20
15/15 [==============================] - 10s 635ms/step - loss: 0.3066 - acc: 0.8952 - val_loss: 0.2637 - val_acc: 0.9033
Epoch 6/20
15/15 [==============================] - 9s 624ms/step - loss: 0.2875 - acc: 0.9001 - val_loss: 0.2420 - val_acc: 0.9133
Epoch 7/20
15/15 [==============================] - 9s 630ms/step - loss: 0.2710 - acc: 0.9057 - val_loss: 0.2350 - val_acc: 0.9102
Epoch 8/20
15/15 [==============================] - 9s 619ms/step - loss: 

### RCNN

In [0]:
def create_rcnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [28]:
%%time

# RCNN, Word Embeddings
tf.keras.backend.clear_session()
classifier = create_bidirectional_rnn(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
20/20 [==============================] - 20s 1s/step - loss: 1.1720 - acc: 0.4778 - val_loss: 0.8795 - val_acc: 0.6781
Epoch 2/20
20/20 [==============================] - 15s 737ms/step - loss: 0.6697 - acc: 0.7627 - val_loss: 0.3682 - val_acc: 0.8750
Epoch 3/20
20/20 [==============================] - 15s 767ms/step - loss: 0.3951 - acc: 0.8652 - val_loss: 0.2989 - val_acc: 0.8958
Epoch 4/20
20/20 [==============================] - 14s 712ms/step - loss: 0.3331 - acc: 0.8842 - val_loss: 0.2719 - val_acc: 0.9027
Epoch 5/20
20/20 [==============================] - 15s 761ms/step - loss: 0.3086 - acc: 0.8920 - val_loss: 0.2577 - val_acc: 0.9085
Epoch 6/20
20/20 [==============================] - 15s 726ms/step - loss: 0.2913 - acc: 0.8978 - val_loss: 0.2602 - val_acc: 0.9106
Epoch 7/20
20/20 [==============================] - 15s 754ms/step - loss: 0.2788 - acc: 0.9010 - val_loss: 0.2493 - val_acc: 0.9116
Epoch 8/20
20/20 [==============================] - 14s 718ms/step - los